In [32]:
%useLatestDescriptors
%use dataframe
%use kandy
%use ktor-client

@file:DependsOn("org.jetbrains.kotlinx:kotlinx-io-core:0.8.0")
@file:DependsOn("org.apache.commons:commons-compress:1.28.0")

In [29]:
private val remoteDataUrl = "https://github.com/ageron/handson-ml/raw/master/datasets"
private val housingUrl = "$remoteDataUrl/housing/housing.tgz"

private val localDataDir = "/Users/markn/IdeaProjects/ml-project/datasets"
private val housingTgzFile = "housing.tgz"
private val housingCsvFile = "housing.csv"

val housingTgzPath = Path("$localDataDir/$housingTgzFile")
val housingCsvPath = Path("$localDataDir/$housingCsvFile")

fun loadHousingTgz() : Path {
    val data = http.get(housingUrl).readBytes()
    SystemFileSystem.createDirectories(Path(localDataDir))
    SystemFileSystem.sink(housingTgzPath).buffered().use {
        it.write(data)
    }
    return housingTgzPath
}

fun extractHousingCsv(path: Path) : Path {
    SystemFileSystem.source(path).buffered().use { source ->
        GzipCompressorInputStream(source.asInputStream()).use { gins ->
            TarArchiveInputStream(gins).use { tais ->
                var entry = tais.nextEntry
                while (entry != null) {
                    if (entry.name == housingCsvFile) {
                        SystemFileSystem.sink(housingCsvPath).buffered().use { sink ->
                            sink.write(tais.readBytes())
                        }
                    }
                    entry = tais.nextEntry
                }
            }
        }
    }
    return housingCsvPath
}

In [30]:
val housingTgz = getHousingTgz()
val housingCsv = extractHousingCsv(housingTgz)
val housing = DataFrame.readCsv(File(housingCsv.toString()))

In [57]:
housing

longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
"-122,230000","37,880000","41,000000","880,000000","129,000000","322,000000","126,000000","8,325200","452600,000000",NEAR BAY
"-122,220000","37,860000","21,000000","7099,000000","1106,000000","2401,000000","1138,000000","8,301400","358500,000000",NEAR BAY
"-122,240000","37,850000","52,000000","1467,000000","190,000000","496,000000","177,000000","7,257400","352100,000000",NEAR BAY
"-122,250000","37,850000","52,000000","1274,000000","235,000000","558,000000","219,000000","5,643100","341300,000000",NEAR BAY
"-122,250000","37,850000","52,000000","1627,000000","280,000000","565,000000","259,000000","3,846200","342200,000000",NEAR BAY
"-122,250000","37,850000","52,000000","919,000000","213,000000","413,000000","193,000000","4,036800","269700,000000",NEAR BAY
"-122,250000","37,840000","52,000000","2535,000000","489,000000","1094,000000","514,000000","3,659100","299200,000000",NEAR BAY
"-122,250000","37,840000","52,000000","3104,000000","687,000000","1157,000000","647,000000","3,120000","241400,000000",NEAR BAY
"-122,260000","37,840000","42,000000","2555,000000","665,000000","1206,000000","595,000000","2,080400","226700,000000",NEAR BAY
"-122,250000","37,840000","52,000000","3549,000000","707,000000","1551,000000","714,000000","3,691200","261100,000000",NEAR BAY


In [59]:
val plots = listOf(
    housing.plot { histogram(longitude) },
    housing.plot { histogram(latitude) },
    housing.plot { histogram(housing_median_age) },

    housing.plot { histogram(total_rooms) },
    housing.plot { histogram(total_bedrooms.map { it ?: -1.0 }.filter { it != -1.0 }) },
    housing.plot { histogram(population) },

    housing.plot { histogram(households) },
    housing.plot { histogram(median_income) },
    housing.plot { histogram(median_house_value) },
)
plotGrid(plots, nCol = 3)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vLpx9E"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":3,
"nrow":3,
"fit":true,
"align":false
},
"figures":[{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[-123.99399999999991,-123.4919999999999,-122.9899999999999,-122.48799999999991,-121.9859999999999,-121.4839999999999,-120.98199999999991,-120.4799999999999,-119.97799999999991,-119.47599999999991,-118.9739999999999,-118.47199999999991,-117.96999999999991,-117.4679999999999,-116.96599999999992,-116.46399999999991,-115.9619999999999,-115.45999999999992,-114.95799999999991,-114.4559999999999],
"count":[151.0,47.0,219.0,2117.0,2792.0,1442.0,752.0,523.0,610.0,617.0,666.0,3366.0,3950.0,1537.0,1518.0,151.0,54.0,97.0,4.0,27.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"longitude",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"202"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[1235.0,495.0,2522.0,6386.0,527.0,248.0,362.0,182.0,467.0,781.0,1823.0,2888.0,1022.0,745.0,380.0,165.0,103.0,219.0,42.0,48.0],
"x":[32.699750000000016,33.17025000000002,33.64075000000002,34.11125000000001,34.581750000000014,35.052250000000015,35.522750000000016,35.99325000000002,36.46375000000002,36.93425000000002,37.404750000000014,37.875250000000015,38.34575000000002,38.81625000000002,39.28675000000002,39.75725000000002,40.22775000000002,40.698250000000016,41.16875000000002,41.63925000000002]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"latitude",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"203"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[62.0,497.0,335.0,675.0,492.0,1226.0,1469.0,1537.0,845.0,1492.0,1578.0,937.0,1638.0,1513.0,1793.0,673.0,1017.0,650.0,620.0,1591.0],
"x":[1.275,3.8249999999999997,6.375,8.924999999999999,11.475,14.024999999999999,16.575,19.125,21.674999999999997,24.225,26.775,29.324999999999996,31.875,34.425,36.974999999999994,39.525,42.074999999999996,44.625,47.175,49.724999999999994]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"housing_median_age",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"spec_id":"204"
},{
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[9157.0,8366.0,1935.0,625.0,250.0,122.0,63.0,40.0,32.0,17.0,12.0,4.0,5.0,2.0,3.0,3.0,2.0,0.0,0.0,2.0],
"x":[982.95,2948.8500000000004,4914.75,6880.650000000001,8846.55,10812.45,12778.350000000002,14744.25,16710.15,18676.050000000003,20641.95,22607.850000000002,24573.75,26539.65,28505.550000000003,30471.45,32437.350000000002,34403.25,36369.15,38335.05]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column"